In [1]:
#download_data_from_blob
import os
from azure.storage.blob import ContainerClient
from azure.storage.blob import BlobClient
from json import loads
from os.path import join
import datetime
from datetime import datetime, timedelta
import _thread

import threading
import time

class myThread (threading.Thread):
    def __init__(self, blob, i, directory_path,cs):
        threading.Thread.__init__(self)
        self.blob = blob
        self.i = i
        self.directory_path = directory_path
        self.cs = cs
    def run(self):
        threadLock.acquire()
        thread_func(self.blob , self.i,self.directory_path , self.cs)
        # Free lock to release next thread
        threadLock.release()

def thread_func(blob , i,directory_path , cs):
        download_path = directory_path+'/entry_'+str(i)+'.json'
        blob = BlobClient.from_connection_string(conn_str=cs, container_name="sampleiothubcontainer",blob_name=blob.name)
        with open(download_path, "wb") as my_blob:       
            blob_data = blob.download_blob()
            blob_data.readinto(my_blob)
threadLock = threading.Lock()
threads = []


class FootPrintData:
    ''' Download footprints data of providers from iothub container '''
    
    def __init__(self):
        file='config/mongo_creds.json'
        pwd= os.getcwd()
        path = join(pwd, file)
        if not os.path.exists(path):
            raise FileNotFoundError('File', path, 'not found')
        f = open(path)
        self.creds = loads(f.read()) 
        self.connection_string = self.creds['connection_string_blob']
        
        
        
    
    
    
    
    def thread_func(self, blob , i,directory_path):
        download_path = directory_path+'/entry_'+str(i)+'.json'
        blob = BlobClient.from_connection_string(conn_str=self.connection_string, container_name="iothubcontainer",blob_name=blob.name)
        with open(download_path, "wb") as my_blob:       
            blob_data = blob.download_blob()
            blob_data.readinto(my_blob)
        
    def get_data(self,timespan=7):
        ''' 
            This method is used to download data, create directory and store the data there. 
                args: 
                    timespan : Number of days' data to download. Here 7 stands for last 7 days' data  
                    
        '''
        try:       
            container = ContainerClient.from_connection_string(conn_str=self.connection_string, container_name="sampleiothubcontainer")
            for number in range(1,timespan+1):
                date = datetime.strftime(datetime.now() - timedelta(number), '%d-%m-%Y')
                print(date)
                blob_list = container.list_blobs(name_starts_with="path/to/provider/footprints/location-history-"+date+"/")
                i=0   
                pwd = os.getcwd()
                directory_path = join(pwd, 'location-history-'+date)
                if not os.path.exists(directory_path):
                    os.mkdir(directory_path)
                    for blob in blob_list:
                        #_thread.start_new_thread( self.thread_func, (blob, i,directory_path, ) )
                        #self.thread_func(blob , i, directory_path)
                        thread = myThread(blob, i, directory_path , self.connection_string)
                        thread.start()
                        threads.append(thread)
                        i= i + 1
                   
                        
        except Exception as ex:
            raise ("Error in downloading blob")
            print(ex)
        for t in threads:
            t.join()
        print("ENDED")
            
                                 

In [2]:
abas = FootPrintData()

In [ ]:
abas.get_data(1)

06-06-2021
